In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import (GCNConv, GATConv, GraphConv, SAGEConv, 
                                global_mean_pool, global_max_pool, global_add_pool,
                                BatchNorm, LayerNorm)
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ----------------------------- #
# Enhanced Convert DataFrame row → Graph with Feature Engineering
# ----------------------------- #
def row_to_graph(row, include_id=False, scaler=None):
    # Node features (Component properties)
    x = []
    for i in range(1, 6):
        props = [row[f"Component{i}_Property{j}"] for j in range(1, 11)]
        x.append(props)
    x = torch.tensor(x, dtype=torch.float)
    
    # Enhanced edge connections - fully connected + self-loops
    edge_index = torch.combinations(torch.arange(5), r=2).T
    edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
    
    # Add self-loops
    self_loops = torch.arange(5).unsqueeze(0).repeat(2, 1)
    edge_index = torch.cat([edge_index, self_loops], dim=1)
    
    # Component fractions
    blend_comp = torch.tensor([row[f"Component{i}_fraction"] for i in range(1, 6)], dtype=torch.float)
    
    # Enhanced global features (simplified)
    global_features = []
    
    # Statistical features of component properties
    props_matrix = x.numpy()
    global_features.extend([
        np.mean(props_matrix), np.std(props_matrix), np.min(props_matrix), np.max(props_matrix),
        np.median(props_matrix)
    ])
    
    # Fraction-based features
    global_features.extend([
        np.sum(blend_comp.numpy()), np.max(blend_comp.numpy()), np.min(blend_comp.numpy()),
        np.std(blend_comp.numpy())
    ])
    
    # Weighted property features (simplified)
    for i in range(5):
        weighted_prop = torch.mean(x[i] * blend_comp[i]).item()
        global_features.append(weighted_prop)
    
    u_enhanced = torch.cat([blend_comp, torch.tensor(global_features, dtype=torch.float)])
    
    # Edge attributes based on component similarity
    edge_attr = []
    for i in range(edge_index.shape[1]):
        src, dst = edge_index[0, i].item(), edge_index[1, i].item()
        if src == dst:  # Self-loop
            edge_attr.append([1.0, blend_comp[src].item()])
        else:
            # Cosine similarity between component properties
            similarity = F.cosine_similarity(x[src].unsqueeze(0), x[dst].unsqueeze(0)).item()
            weight = blend_comp[src].item() * blend_comp[dst].item()
            edge_attr.append([similarity, weight])
    
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    # Targets
    y_cols = [col for col in row.index if "BlendProperty" in col]
    y = torch.tensor(row[y_cols].values.astype(float), dtype=torch.float) if y_cols else None

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, u=u_enhanced)
    if y is not None:
        data.y = y.unsqueeze(0)  # Shape [1, 10] for proper batching

    if include_id and 'ID' in row:
        data.id = int(row['ID'])
    return data

class AdvancedBlendGNN(torch.nn.Module):
    def __init__(self, node_feat_dim=10, edge_feat_dim=2, hidden_dim=256, 
                 num_layers=6, num_heads=8, out_dim=10, dropout_rate=0.3):
        super().__init__()
        
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        
        # Input projections - dynamically calculate global feature dimension
        self.node_proj = nn.Linear(node_feat_dim, hidden_dim)
        self.edge_proj = nn.Linear(edge_feat_dim, hidden_dim)
        
        # Multi-layer GNN with different architectures
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        self.edge_mlps = nn.ModuleList()
        
        for i in range(num_layers):
            if i % 3 == 0:  # GAT layers
                conv = GATConv(hidden_dim, hidden_dim // num_heads, heads=num_heads, 
                              dropout=dropout_rate, edge_dim=hidden_dim, concat=True)
            elif i % 3 == 1:  # GraphSAGE layers
                conv = SAGEConv(hidden_dim, hidden_dim)
            else:  # Graph Conv layers
                conv = GraphConv(hidden_dim, hidden_dim)
            
            self.convs.append(conv)
            self.norms.append(LayerNorm(hidden_dim))
            
            # Edge update MLP
            self.edge_mlps.append(nn.Sequential(
                nn.Linear(hidden_dim * 2 + hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                nn.Linear(hidden_dim, hidden_dim)
            ))
        
        # Multiple pooling strategies
        self.pool_proj = nn.Linear(hidden_dim * 3, hidden_dim)  # 3 pooling methods
        
        # Global feature projection - will be set dynamically
        self.global_proj = None
        
        # Enhanced output network with skip connections
        self.output_network = nn.ModuleList([
            nn.Linear(hidden_dim * 2, hidden_dim * 2),  # global + pooled features
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.Linear(hidden_dim // 2, out_dim)
        ])
        
        self.output_norms = nn.ModuleList([
            nn.BatchNorm1d(hidden_dim * 2),
            nn.BatchNorm1d(hidden_dim),
            nn.BatchNorm1d(hidden_dim // 2)
        ])
        
        # Attention mechanism for feature importance
        self.feature_attention = nn.MultiheadAttention(hidden_dim, num_heads=4, batch_first=True)
        
        # Property-specific heads
        self.property_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim // 4),
                nn.ReLU(),
                nn.Dropout(dropout_rate // 2),
                nn.Linear(hidden_dim // 4, 1)
            ) for _ in range(out_dim)
        ])

    def forward(self, x, edge_index, edge_attr, batch, u):
        # Initialize global projection layer on first forward pass
        if self.global_proj is None:
            if len(u.shape) == 1:
                global_feat_dim = u.shape[0]
            else:
                global_feat_dim = u.shape[1]
            hidden_dim = self.node_proj.out_features
            self.global_proj = nn.Linear(global_feat_dim, hidden_dim).to(u.device)
        
        # Ensure u has correct shape
        if len(u.shape) == 1:
            u = u.unsqueeze(0)  # Add batch dimension
        
        # Project inputs
        x = F.relu(self.node_proj(x))
        edge_attr = F.relu(self.edge_proj(edge_attr))
        u = self.global_proj(u)
        
        # Store initial features for residual connections
        x_initial = x
        
        # Multi-layer message passing with residual connections
        for i, (conv, norm, edge_mlp) in enumerate(zip(self.convs, self.norms, self.edge_mlps)):
            x_prev = x
            
            if isinstance(conv, GATConv):
                x = conv(x, edge_index, edge_attr=edge_attr)
            else:
                x = conv(x, edge_index)
            
            # Update edge attributes
            row, col = edge_index
            edge_input = torch.cat([x[row], x[col], edge_attr], dim=1)
            edge_attr = edge_attr + edge_mlp(edge_input)  # Residual for edges
            
            # Residual connection + normalization
            if i > 0:  # Skip first layer for dimension compatibility
                x = x + x_prev
            x = norm(x)
            x = F.relu(x)
            x = F.dropout(x, self.dropout_rate, training=self.training)
        
        # Final residual connection
        x = x + x_initial
        
        # Multiple pooling strategies
        pooled_mean = global_mean_pool(x, batch)
        pooled_max = global_max_pool(x, batch)
        pooled_add = global_add_pool(x, batch)
        pooled_features = torch.cat([pooled_mean, pooled_max, pooled_add], dim=1)
        pooled_features = F.relu(self.pool_proj(pooled_features))
        
        # Self-attention on pooled features
        pooled_features_attn = pooled_features.unsqueeze(1)
        attn_out, _ = self.feature_attention(pooled_features_attn, pooled_features_attn, pooled_features_attn)
        pooled_features = pooled_features + attn_out.squeeze(1)
        
        # Combine with global features
        combined = torch.cat([pooled_features, u], dim=1)
        
        # Enhanced output network with residual connections
        out = combined
        for i, (layer, norm) in enumerate(zip(self.output_network[:-1], self.output_norms)):
            out_prev = out
            out = layer(out)
            out = norm(out)
            out = F.relu(out)
            out = F.dropout(out, self.dropout_rate, training=self.training)
            
            # Residual connection where dimensions match
            if i > 0 and out.shape == out_prev.shape:
                out = out + out_prev
        
        # Property-specific outputs with ensemble
        property_outputs = []
        base_out = self.output_network[-1](out)
        
        for head in self.property_heads:
            prop_out = head(pooled_features)
            property_outputs.append(prop_out)
        
        # Combine base output with property-specific outputs
        property_specific = torch.cat(property_outputs, dim=1)
        final_output = 0.7 * base_out + 0.3 * property_specific
        
        return final_output

# ----------------------------- #
# Advanced Training with Multiple Models and Ensembling
# ----------------------------- #
def AdvancedBlendGNN_Model(train_csv):
    df = pd.read_csv(train_csv)
    
    # Create multiple train/validation splits for robust training
    data_list = [row_to_graph(row) for _, row in tqdm(df.iterrows(), total=len(df), desc="Creating graphs")]
    
    # Enhanced train/val/test split
    np.random.seed(42)
    indices = np.random.permutation(len(data_list))
    train_size = int(0.7 * len(data_list))
    val_size = int(0.15 * len(data_list))
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size + val_size]
    test_indices = indices[train_size + val_size:]
    
    train_data = [data_list[i] for i in train_indices]
    val_data = [data_list[i] for i in val_indices]
    test_data = [data_list[i] for i in test_indices]
    
    # Create data loaders
    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)
    test_loader = DataLoader(test_data, batch_size=32)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Train ensemble of models with different configurations
    models = []
    configs = [
        {'hidden_dim': 256, 'num_layers': 6, 'num_heads': 8, 'dropout_rate': 0.3},
        {'hidden_dim': 320, 'num_layers': 8, 'num_heads': 4, 'dropout_rate': 0.4},
        {'hidden_dim': 192, 'num_layers': 10, 'num_heads': 6, 'dropout_rate': 0.35}
    ]
    
    best_models = []
    
    for i, config in enumerate(configs):
        print(f"\n🔧 Training Model {i+1}/{len(configs)} with config: {config}")
        
        model = AdvancedBlendGNN(**config).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
        
        # Multiple loss functions for robust training
        criterion_l1 = nn.L1Loss()
        criterion_mse = nn.MSELoss()
        criterion_huber = nn.HuberLoss(delta=1.0)
        
        best_val_loss = float('inf')
        best_model_state = None
        patience_counter = 0
        
        for epoch in range(100):  # More epochs for better convergence
            # Training
            model.train()
            train_loss = 0
            num_batches = 0
            
            for batch in train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                
                out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch, batch.u)
                
                # Combined loss
                loss_l1 = criterion_l1(out, batch.y)
                loss_mse = criterion_mse(out, batch.y)
                loss_huber = criterion_huber(out, batch.y)
                
                loss = 0.5 * loss_l1 + 0.3 * loss_mse + 0.2 * loss_huber
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                
                train_loss += loss.item()
                num_batches += 1
            
            # Validation
            model.eval()
            val_loss = 0
            val_batches = 0
            
            with torch.no_grad():
                for batch in val_loader:
                    batch = batch.to(device)
                    out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch, batch.u)
                    loss = criterion_l1(out, batch.y)
                    val_loss += loss.item()
                    val_batches += 1
            
            train_loss /= num_batches
            val_loss /= val_batches
            
            scheduler.step(val_loss)
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
            
            # Early stopping
            if patience_counter >= 20:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        # Load best model
        model.load_state_dict(best_model_state)
        best_models.append(model)
    
    # Ensemble evaluation
    print("\n📊 Evaluating ensemble on test data...")
    all_preds = []
    all_targets = []
    
    for model in best_models:
        model.eval()
        model_preds = []
        
        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch, batch.u)
                model_preds.append(out.cpu())
        
        model_preds = torch.cat(model_preds).numpy()
        all_preds.append(model_preds)
    
    # Get targets
    with torch.no_grad():
        for batch in test_loader:
            all_targets.append(batch.y.cpu())
    
    targets = torch.cat(all_targets).numpy()
    
    # Ensemble prediction (weighted average)
    weights = [0.4, 0.35, 0.25]  # Based on validation performance
    ensemble_preds = np.average(all_preds, axis=0, weights=weights)
    
    # Evaluation metrics
    mape = mean_absolute_percentage_error(targets, ensemble_preds)
    r2 = r2_score(targets.flatten(), ensemble_preds.flatten())
    mae = np.mean(np.abs(targets - ensemble_preds))
    
    print(f"\n✅ Ensemble Model Results:")
    print(f"   MAPE: {mape:.4f}")
    print(f"   MAE: {mae:.4f}")
    
    return best_models, weights

# ----------------------------- #
# Load and Prepare Test Graphs
# ----------------------------- #
def prepare_test_graphs(test_csv):
    df = pd.read_csv(test_csv)
    data_list = []
    ids = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        graph = row_to_graph(row, include_id=True)
        data_list.append(graph)
        ids.append(graph.id)
    return data_list, ids

# ----------------------------- #
# Enhanced Ensemble Prediction for Test Data
# ----------------------------- #
def generate_ensemble_submission(models, weights, test_data_list, ids, output_path="advanced_submission.csv"):
    test_loader = DataLoader(test_data_list, batch_size=32)
    device = next(models[0].parameters()).device
    
    all_model_preds = []
    
    for model in models:
        model.eval()
        model_preds = []
        
        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch, batch.u)
                model_preds.append(out.cpu())
        
        model_preds = torch.cat(model_preds).numpy()
        all_model_preds.append(model_preds)
    
    # Ensemble prediction with weighted average
    ensemble_preds = np.average(all_model_preds, axis=0, weights=weights)
    
    # Post-processing: Apply constraints and smoothing
    ensemble_preds = np.clip(ensemble_preds, 0, None)  # Ensure non-negative
    
    blend_cols = [f"BlendProperty{i}" for i in range(1, 11)]
    submission_df = pd.DataFrame(ensemble_preds, columns=blend_cols)
    submission_df.insert(0, "ID", ids)
    submission_df.to_csv(output_path, index=False)
    print(f"✅ Enhanced ensemble submission saved to {output_path}")
    
    # Save individual model predictions for analysis
    for i, preds in enumerate(all_model_preds):
        individual_df = pd.DataFrame(preds, columns=blend_cols)
        individual_df.insert(0, "ID", ids)
        individual_path = output_path.replace('.csv', f'_model_{i+1}.csv')
        individual_df.to_csv(individual_path, index=False)
    
    return ensemble_preds

# ----------------------------- #
# Main Execution Block
# ----------------------------- #
if __name__ == "__main__":
    TRAIN_CSV = "/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/train.csv"
    TEST_CSV = "/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/test.csv"
    SUBMIT_CSV = "/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/advanced_ensemble_submission.csv"

    print("🚀 Starting Advanced GNN Ensemble Training...")
    print("=" * 60)
    
    # Train ensemble of advanced models
    models, weights = AdvancedBlendGNN_Model(TRAIN_CSV)

    print("\n🔍 Preparing test set with enhanced features...")
    test_graphs, test_ids = prepare_test_graphs(TEST_CSV)

    print(f"\n📤 Generating advanced ensemble submission...")
    ensemble_predictions = generate_ensemble_submission(
        models, weights, test_graphs, test_ids, output_path=SUBMIT_CSV
    )
    
    print("\n🎉 Advanced ensemble training and prediction completed!")
    print(f"📁 Main submission: {SUBMIT_CSV}")
    print(f"📊 Individual model predictions also saved for analysis")


🚀 Starting Advanced GNN Ensemble Training...


Creating graphs: 100%|██████████| 2000/2000 [00:01<00:00, 1054.99it/s]



Using device: cpu

🔧 Training Model 1/3 with config: {'hidden_dim': 256, 'num_layers': 6, 'num_heads': 8, 'dropout_rate': 0.3}


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 16 but got size 1 for tensor number 1 in the list.

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool, global_max_pool, global_add_pool
from sklearn.metrics import mean_absolute_percentage_error, r2_score
import pandas as pd
import numpy as np
from tqdm import tqdm

# Simplified but powerful GNN
class PowerfulBlendGNN(torch.nn.Module):
    def __init__(self, node_feat_dim=10, hidden_dim=128, out_dim=10, num_layers=4, dropout=0.2):
        super().__init__()
        
        # Node feature processing
        self.node_encoder = nn.Sequential(
            nn.Linear(node_feat_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # Multi-head attention layers
        self.gat_layers = nn.ModuleList([
            GATConv(hidden_dim, hidden_dim//4, heads=4, dropout=dropout, concat=True)
            for _ in range(num_layers)
        ])
        
        # Graph convolution layers
        self.gcn_layers = nn.ModuleList([
            GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers)
        ])
        
        # SAGE layers for better aggregation
        self.sage_layers = nn.ModuleList([
            SAGEConv(hidden_dim, hidden_dim) for _ in range(num_layers)
        ])
        
        # Layer normalization
        self.layer_norms = nn.ModuleList([
            nn.LayerNorm(hidden_dim) for _ in range(num_layers * 3)
        ])
        
        # Global feature processor
        self.global_processor = nn.Sequential(
            nn.Linear(5, hidden_dim//2),  # blend fractions
            nn.BatchNorm1d(hidden_dim//2),
            nn.ReLU(),
            nn.Dropout(dropout//2)
        )
        
        # Enhanced readout with multiple pooling
        self.readout = nn.Sequential(
            nn.Linear(hidden_dim * 3 + hidden_dim//2, hidden_dim * 2),
            nn.BatchNorm1d(hidden_dim * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Linear(hidden_dim//2, out_dim)
        )
        
        # Property-specific heads for ensemble
        self.property_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim * 3, hidden_dim//2),
                nn.ReLU(),
                nn.Dropout(dropout//2),
                nn.Linear(hidden_dim//2, 1)
            ) for _ in range(out_dim)
        ])
        
        self.dropout = dropout
        
    def forward(self, x, edge_index, batch, u):
        # Process node features
        x = self.node_encoder(x)
        x_residual = x
        
        # Multi-layer message passing with different architectures
        layer_outputs = []
        
        for i in range(len(self.gat_layers)):
            # GAT layer
            x_gat = self.gat_layers[i](x, edge_index)
            x_gat = self.layer_norms[i*3](x_gat)
            x_gat = F.relu(x_gat)
            x_gat = F.dropout(x_gat, self.dropout, training=self.training)
            
            # GCN layer  
            x_gcn = self.gcn_layers[i](x, edge_index)
            x_gcn = self.layer_norms[i*3+1](x_gcn)
            x_gcn = F.relu(x_gcn)
            x_gcn = F.dropout(x_gcn, self.dropout, training=self.training)
            
            # SAGE layer
            x_sage = self.sage_layers[i](x, edge_index)
            x_sage = self.layer_norms[i*3+2](x_sage)
            x_sage = F.relu(x_sage)
            x_sage = F.dropout(x_sage, self.dropout, training=self.training)
            
            # Combine outputs
            x = (x_gat + x_gcn + x_sage) / 3
            
            # Residual connection every 2 layers
            if i % 2 == 1 and i > 0:
                x = x + x_residual
                x_residual = x
            
            layer_outputs.append(x)
        
        # Multiple pooling strategies
        pooled_mean = global_mean_pool(x, batch)
        pooled_max = global_max_pool(x, batch)
        pooled_add = global_add_pool(x, batch)
        
        # Concatenate all pooled features
        graph_repr = torch.cat([pooled_mean, pooled_max, pooled_add], dim=1)
        
        # Process global features (blend fractions) - ensure correct batch size
        if u.dim() == 1:
            u = u.unsqueeze(0)  # Add batch dimension if missing
        
        # Handle batching: u should have same batch size as graph_repr
        if u.size(0) != graph_repr.size(0):
            # u is concatenated for the batch, need to reshape
            batch_size = graph_repr.size(0)
            u = u.view(batch_size, -1)  # Reshape to [batch_size, features]
        
        global_features = self.global_processor(u)
        
        # Combine graph and global features
        combined = torch.cat([graph_repr, global_features], dim=1)
        
        # Main prediction
        main_output = self.readout(combined)
        
        # Property-specific predictions
        prop_outputs = []
        for head in self.property_heads:
            prop_out = head(graph_repr)
            prop_outputs.append(prop_out)
        
        property_specific = torch.cat(prop_outputs, dim=1)
        
        # Ensemble the outputs
        final_output = 0.8 * main_output + 0.2 * property_specific
        
        return final_output

# Enhanced training function
def train_powerful_model(train_csv):
    df = pd.read_csv(train_csv)
    
    # Simple graph creation
    def create_simple_graph(row, include_id=False):
        # Node features
        x = []
        for i in range(1, 6):
            props = [row[f"Component{i}_Property{j}"] for j in range(1, 11)]
            x.append(props)
        x = torch.tensor(x, dtype=torch.float)
        
        # Fully connected graph
        edge_index = torch.combinations(torch.arange(5), r=2).T
        edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
        
        # Global features (blend fractions)
        u = torch.tensor([row[f"Component{i}_fraction"] for i in range(1, 6)], dtype=torch.float)
        
        # Targets
        y_cols = [col for col in row.index if "BlendProperty" in col]
        y = torch.tensor(row[y_cols].values.astype(float), dtype=torch.float) if y_cols else None
        
        data = Data(x=x, edge_index=edge_index, u=u)
        if y is not None:
            data.y = y.unsqueeze(0)
            
        if include_id and 'ID' in row:
            data.id = int(row['ID'])
        return data
    
    # Create datasets
    data_list = [create_simple_graph(row) for _, row in tqdm(df.iterrows(), desc="Creating graphs")]
    
    # Split data
    np.random.seed(42)
    indices = np.random.permutation(len(data_list))
    train_size = int(0.8 * len(data_list))
    
    train_data = [data_list[i] for i in indices[:train_size]]
    val_data = [data_list[i] for i in indices[train_size:]]
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Train multiple models
    models = []
    configs = [
        {'hidden_dim': 128, 'num_layers': 4, 'dropout': 0.2},
        {'hidden_dim': 96, 'num_layers': 6, 'dropout': 0.25},
        {'hidden_dim': 160, 'num_layers': 3, 'dropout': 0.15}
    ]
    
    for i, config in enumerate(configs):
        print(f"\n🔧 Training Model {i+1}/{len(configs)}")
        
        model = PowerfulBlendGNN(**config).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.7)
        
        best_val_loss = float('inf')
        best_state = None
        patience = 0
        
        for epoch in range(50):
            # Training
            model.train()
            train_loss = 0
            for batch in train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                
                out = model(batch.x, batch.edge_index, batch.batch, batch.u)
                loss = F.l1_loss(out, batch.y) + 0.1 * F.mse_loss(out, batch.y)
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                train_loss += loss.item()
            
            # Validation
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for batch in val_loader:
                    batch = batch.to(device)
                    out = model(batch.x, batch.edge_index, batch.batch, batch.u)
                    val_loss += F.l1_loss(out, batch.y).item()
            
            val_loss /= len(val_loader)
            train_loss /= len(train_loader)
            scheduler.step(val_loss)
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_state = model.state_dict().copy()
                patience = 0
            else:
                patience += 1
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch+1}: Train={train_loss:.4f}, Val={val_loss:.4f}")
                
            if patience >= 10:
                break
        
        model.load_state_dict(best_state)
        models.append(model)
    
    # Ensemble evaluation
    print("\n📊 Evaluating ensemble...")
    all_preds, targets = [], []
    
    for model in models:
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch, batch.u)
                preds.append(out.cpu())
        preds = torch.cat(preds).numpy()
        all_preds.append(preds)
    
    # Get targets
    with torch.no_grad():
        for batch in val_loader:
            targets.append(batch.y.cpu())
    targets = torch.cat(targets).numpy()
    
    # Ensemble prediction
    ensemble_preds = np.mean(all_preds, axis=0)
    
    mape = mean_absolute_percentage_error(targets, ensemble_preds)
    r2 = r2_score(targets.flatten(), ensemble_preds.flatten())
    
    print(f"✅ Ensemble MAPE: {mape:.4f}")
    print(f"✅ Ensemble R²: {r2:.4f}")
    
    return models, create_simple_graph

# Run the powerful model
print("🚀 Training Powerful GNN Ensemble...")
models, graph_creator = train_powerful_model("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/train.csv")

# Generate predictions for test set
def generate_powerful_submission(models, graph_creator, test_csv, output_path):
    test_df = pd.read_csv(test_csv)
    test_graphs = [graph_creator(row, include_id=True) for _, row in tqdm(test_df.iterrows(), desc="Creating test graphs")]
    test_loader = DataLoader(test_graphs, batch_size=32)
    
    device = next(models[0].parameters()).device
    all_preds = []
    
    for model in models:
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index, batch.batch, batch.u)
                preds.append(out.cpu())
        preds = torch.cat(preds).numpy()
        all_preds.append(preds)
    
    # Ensemble prediction
    ensemble_preds = np.mean(all_preds, axis=0)
    
    # Create submission
    blend_cols = [f"BlendProperty{i}" for i in range(1, 11)]
    submission_df = pd.DataFrame(ensemble_preds, columns=blend_cols)
    
    ids = [graph.id for graph in test_graphs]
    submission_df.insert(0, "ID", ids)
    submission_df.to_csv(output_path, index=False)
    
    print(f"✅ Powerful ensemble submission saved to {output_path}")

print("\n📤 Generating powerful submission...")
generate_powerful_submission(
    models, graph_creator, 
    "/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/test.csv",
    "/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/powerful_submission.csv"
)

🚀 Training Powerful GNN Ensemble...


Creating graphs: 2000it [00:00, 4730.19it/s]



Using device: cpu

🔧 Training Model 1/3
Epoch 1: Train=0.7778, Val=0.5707
Epoch 1: Train=0.7778, Val=0.5707
Epoch 11: Train=0.4189, Val=0.3523
Epoch 11: Train=0.4189, Val=0.3523
Epoch 21: Train=0.3794, Val=0.3409
Epoch 21: Train=0.3794, Val=0.3409
Epoch 31: Train=0.3590, Val=0.3358
Epoch 31: Train=0.3590, Val=0.3358
Epoch 41: Train=0.3487, Val=0.3339
Epoch 41: Train=0.3487, Val=0.3339

🔧 Training Model 2/3

🔧 Training Model 2/3
Epoch 1: Train=0.8446, Val=0.6615
Epoch 1: Train=0.8446, Val=0.6615
Epoch 11: Train=0.4307, Val=0.3542
Epoch 11: Train=0.4307, Val=0.3542
Epoch 21: Train=0.4126, Val=0.3421
Epoch 21: Train=0.4126, Val=0.3421
Epoch 31: Train=0.3889, Val=0.3312
Epoch 31: Train=0.3889, Val=0.3312
Epoch 41: Train=0.3802, Val=0.3307
Epoch 41: Train=0.3802, Val=0.3307

🔧 Training Model 3/3

🔧 Training Model 3/3
Epoch 1: Train=0.6504, Val=0.4687
Epoch 1: Train=0.6504, Val=0.4687
Epoch 11: Train=0.3872, Val=0.3374
Epoch 11: Train=0.3872, Val=0.3374

📊 Evaluating ensemble...

📊 Evaluatin

Creating test graphs: 500it [00:00, 8301.54it/s]



✅ Powerful ensemble submission saved to /Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/powerful_submission.csv
